In [1]:
from sklearn.preprocessing import StandardScaler  
from sklearn import svm 
import numpy as np
from amrafile import amrafile as af
from amracommon.analysis.registration import normalized_cross_correlation
import matplotlib.pyplot as plt

trainTargets = ["00002","00003","00004","00005","00006","00007","00008","00009",
   "0000A","0000B","0000C","0000D","0000E","0000F","00010","00011",
   "00012","00013","00014","00015","00016","00017","00018","00019",
   "00055","000B2","0010F","0016D","001C9","00226","00288","002E4",
   "00341","0039E","003FB","00056","000B3","00110","0016E","001CA", 
   "00227","00289","002E5","00342","0039F","003FC","00057","000B4",
   "00111","0016F","001CB","00228","0028A","002E6","00343","003A0",
   "003FD","00058","000B5","00112","00170","001CC","00229","0028B",
   "002E7","00344","003A1","003FE","00059","000B6","00113","00171",
   "001CD","0022A","0028C","002E8","00345","003A2","003FF","0005A",
   "000B7","00114","00172","001CE","0022C","0028D","002E9","00346",
   "003A3","00400","0005B","000B8","00115","00173","001CF","0022D",
   "0028E","002EA","00347","003A4","00401","0005C","000B9","00116",
   "00174","001D0","0022E","0028F","002EB","00348","003A5","00402"]

testTargets =  ["0013A","0013B","0013C","0013D","0013E","0013F","00130","00131",
     "00132","00133","00134","00135","00136","00137","0014A","0014B",
     "0014D","0014F","00140","00141","00142","00143","00144","00145",
     "00146","00147","0015A","0015B","0015C","0015D","0015E","0015F",
     "00152","00154","00155","00156","00157","00158","00159"]

features, groundTruth = np.array([]), np.array([])

for target in trainTargets:

    signal = af.parse('/moria/data/DB/0030/'+target+'/wholebody_normalized_water_1_'+target+'.amra')
    
    rf = signal.get_poi('RightFemur')
    lf = signal.get_poi('LeftFemur')
    
    T9 = signal.get_poi('T9')
    
    meanFemur = np.round((lf[0]+rf[0])/2)
    
    voxelSizes = np.array([signal.voxel_size_z, signal.voxel_size_y, signal.voxel_size_x])
    
    femurDist = abs(np.sqrt(np.sum(((np.array(rf) - np.array(lf))*voxelSizes)**2)))
    
    femurHeight = meanFemur*signal.voxel_size_z
            
    femurT9Dist = (T9[0] - meanFemur)*signal.voxel_size_z
        
    tempFeatures = np.array([femurDist, signal.patient_size*1000, signal.patient_weight])
    
    features = np.vstack([features, tempFeatures]) if features.size else tempFeatures
    groundTruth = np.hstack([groundTruth, femurT9Dist]) if groundTruth.size else femurT9Dist

    
scaler = StandardScaler()  
scaler.fit(features)
                             
X_train = scaler.transform(features)
y_train = groundTruth
                           
clf = svm.SVR()
clf.fit(X_train, y_train)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [2]:
features, groundTruth = np.array([]), np.array([])

for target in testTargets:

    signal = af.parse('/moria/data/DB/0030/'+target+'/wholebody_normalized_water_1_'+target+'.amra')
    
    rf = signal.get_poi('RightFemur')
    lf = signal.get_poi('LeftFemur')
    
    T9 = signal.get_poi('T9')
    
    meanFemur = np.round((lf[0]+rf[0])/2)
    
    voxelSizes = np.array([signal.voxel_size_z, signal.voxel_size_y, signal.voxel_size_x])
    
    femurDist = abs(np.sqrt(np.sum(((np.array(rf) - np.array(lf))*voxelSizes)**2)))
    
    femurHeight = meanFemur*signal.voxel_size_z
            
    femurT9Dist = (T9[0] - meanFemur)*signal.voxel_size_z
        
    tempFeatures = np.array([femurDist, signal.patient_size*1000, signal.patient_weight])
    
    features = np.vstack([features, tempFeatures]) if features.size else tempFeatures
    groundTruth = np.hstack([groundTruth, femurT9Dist]) if groundTruth.size else femurT9Dist
    
X_test = scaler.transform(features)
prediction = clf.predict(X_test)
                           
abs(prediction - groundTruth)/signal.voxel_size_z

array([  2.46857106,   5.25829065,   1.05941052,   0.50147658,
         5.164591  ,   5.30216626,   7.08870328,   6.63025069,
         4.13495707,   7.25115985,   9.61244658,   8.40578722,
         0.33365708,   0.0518524 ,   4.4694647 ,   5.54374907,
         7.21742137,   2.87098236,   7.15727484,   0.88014298,
         2.00664135,  11.7135592 ,  12.29455744,   4.53825487,
         2.65882043,   4.89065386,   3.977102  ,   4.62531467,
         5.08511503,   3.90591728,   1.51382574,   9.43162313,
        11.22409846,   3.97909277,   0.33056498,  10.40048071,
         3.91696839,   4.69980963,   3.149519  ])

In [60]:
signal = af.parse('/moria/data/DB/0030/0013A/wholebody_normalized_water_1_0013A.amra')

In [61]:
T9 = signal.get_poi('T9')
LF = signal.get_poi('LeftFemur')
RF = signal.get_poi('RightFemur')

In [4]:
np.std((abs(prediction - groundTruth)/signal.voxel_size_z).astype(int))

3.2643754795568305

In [6]:
groundTruth

array([ 374.49186992,  371.49593496,  371.49593496,  356.51626016,
        353.5203252 ,  380.48373984,  383.4796748 ,  335.54471545,
        368.5       ,  389.47154472,  323.56097561,  326.55691057,
        371.49593496,  368.5       ,  371.49593496,  374.49186992,
        350.52439024,  353.5203252 ,  332.54878049,  365.50406504,
        353.5203252 ,  320.56504065,  401.45528455,  386.47560976,
        365.50406504,  386.47560976,  374.49186992,  374.49186992,
        344.53252033,  368.5       ,  356.51626016,  395.46341463,
        326.55691057,  341.53658537,  365.50406504,  323.56097561,
        344.53252033,  341.53658537,  362.50813008])